In [24]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import pandas as pd
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DEll\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DEll\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DEll\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
blog_text = input("ENTER THE BLOG TEXT: ")


ENTER THE BLOG TEXT:  During the last year, we have been piloting the Unspoken Smiles after-school program with a small group of students at Ember Charter Schools in Brooklyn and have seen dramatic improvements with all of our students showing an increase in their brushing routines at home and a decrease of plaques and tooth decay.    While we have seen measurable success, with school closures due to COVIT-19, the lives and routines of 1.1 million NYC children, have no doubt prompt broader upheaval in a moment of profound anxiety for New Yorkers.    That's why we launched the Unspoken Smiles Virtual classroom, a new program that connects career role models over video chat to local primary schools throughout the five boroughs on topics related to oral health and well-being. 


In [26]:
sentence_list=nltk.sent_tokenize(blog_text)
print(sentence_list)
print(len(sentence_list))

[' During the last year, we have been piloting the Unspoken Smiles after-school program with a small group of students at Ember Charter Schools in Brooklyn and have seen dramatic improvements with all of our students showing an increase in their brushing routines at home and a decrease of plaques and tooth decay.', 'While we have seen measurable success, with school closures due to COVIT-19, the lives and routines of 1.1 million NYC children, have no doubt prompt broader upheaval in a moment of profound anxiety for New Yorkers.', "That's why we launched the Unspoken Smiles Virtual classroom, a new program that connects career role models over video chat to local primary schools throughout the five boroughs on topics related to oral health and well-being."]
3


In [27]:
#creating copy of sentence tokenizing list
change_sentence_list = sentence_list.copy()


# removing all punctuations
for i in range(0,len(change_sentence_list)):
    change_sentence_list[i] = re.sub(r'[^\w\s]','',change_sentence_list[i])
    

# stopwords list and lemmatizer
stopwords_list=stopwords.words('english')
lemmatizer = WordNetLemmatizer()


#splitting sentences into words and storing them in place of sentences as list of words
for i in range(0,len(change_sentence_list)):
    change_sentence_list[i] = change_sentence_list[i].split()

# a list to store unique important words
unique_words=[]

# lemmitizing , stemming and removing stopwords and digits from words lists
for i in range(0,len(change_sentence_list)):
    b=[]
    for j in range(0,len(change_sentence_list[i])):
        change_sentence_list[i][j]=change_sentence_list[i][j].lower()
        
        change_sentence_list[i][j]=lemmatizer.lemmatize(change_sentence_list[i][j]) 
        
        if change_sentence_list[i][j] not in stopwords_list and change_sentence_list[i][j].isnumeric()!=True:
            b.append(change_sentence_list[i][j])
            unique_words.append(change_sentence_list[i][j])
    
    change_sentence_list[i]=b        


#removing the words that are repeated by converting list to set   
unique_words = list(set(unique_words))
unique_words.sort()





In [28]:
# BAG OF WORDS
bag_of_words={}
for j in range(0,len(change_sentence_list)):
    b=[]
    for i in unique_words:
        if i in change_sentence_list[j]:
            b.append(change_sentence_list[j].count(i))
        else:
            b.append(0)
    bag_of_words[j] = b
    
bow_df=pd.DataFrame(bag_of_words,index=unique_words)
bow_df

,0,1,2
afterschool,1,0,0
anxiety,0,1,0
borough,0,0,1
broader,0,1,0
brooklyn,1,0,0
...,...,...,...
video,0,0,1
virtual,0,0,1
wellbeing,0,0,1
year,1,0,0


In [29]:
# Tf-IDF
document_freq=[]
for i in range(0,len(unique_words)):
    b=0
    for j in change_sentence_list:
        if unique_words[i] in j:
            b=b+1
    document_freq.append(math.log(len(change_sentence_list)/b)+1)   

tf_idf={}
for i in range(0,len(change_sentence_list)):
    tf_idf[i] = bow_df[i]*document_freq

tfidf_df = pd.DataFrame(tf_idf)
tfidf_df

,0,1,2
afterschool,2.098612,0.000000,0.000000
anxiety,0.000000,2.098612,0.000000
borough,0.000000,0.000000,2.098612
broader,0.000000,2.098612,0.000000
brooklyn,2.098612,0.000000,0.000000
...,...,...,...
video,0.000000,0.000000,2.098612
virtual,0.000000,0.000000,2.098612
wellbeing,0.000000,0.000000,2.098612
year,2.098612,0.000000,0.000000


In [30]:
# COSINE SIMILARITY MATRIX

sim_matrix={}
for i in range(0,len(change_sentence_list)):
    b=[]
    for j in range(0,len(change_sentence_list)):
        a = np.dot(tfidf_df[i],tfidf_df[j])
        a = a/(np.linalg.norm(tfidf_df[i])*np.linalg.norm(tfidf_df[j]))
        b.append(a)
    sim_matrix[i]=b
sim_matrix=pd.DataFrame(sim_matrix)
sim_matrix


,0,1,2
0,1.000000,0.051683,0.066403
1,0.051683,1.000000,0.033405
2,0.066403,0.033405,1.000000


In [38]:
# A dictionary which stores the sentence index and the points sentence has by 
sent_ranks={}
for i in range(0,len(change_sentence_list)):
    a=0
    for j in range(0,len(unique_words)):
        a=a+bow_df[i][j]
    sent_ranks[i] = a
print(sent_ranks)  

{0: 27, 1: 21, 2: 25}


In [39]:
# Converting the dictionary to list and sorting it by points
sent_ranks= sorted(sent_ranks.items(), key=lambda x: x[1],reverse=True)    
print(sent_ranks)

# Making a list of important sentences and sorting them by the sentence index
imp_sent=[]
num_sentences = int(input("Enter the number of sentences you want to see in summary: "))
if(num_sentences<=len(sentence_list)):
    for i in range(0,num_sentences):
        imp_sent.append(sent_ranks[i][0])
    imp_sent.sort()
    for i in range(0,num_sentences):
        print("\n",sentence_list[imp_sent[i]])
else:
    print("Text has less number of sentences than demanded in the summary!!!!")

[(0, 27), (2, 25), (1, 21)]
Enter the number of sentences you want to see in summary: 1

  During the last year, we have been piloting the Unspoken Smiles after-school program with a small group of students at Ember Charter Schools in Brooklyn and have seen dramatic improvements with all of our students showing an increase in their brushing routines at home and a decrease of plaques and tooth decay.
